## Import from libraries

In [1]:
pip install google-colab

In [1]:
import cv2           # extração dos pixels;
import numpy as np
import os
import zipfile
from google.colab.patches import cv2_imshow
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Extraction of pixels from images

In [1]:
path = '../input/neural-networks-homer-and-bart-classification/homer_bart_1'

In [1]:
directory = '../input/neural-networks-homer-and-bart-classification/homer_bart_1'
files = [os.path.join(directory, f) for f in sorted(os.listdir(directory))]




**THIS IS ANOTHER ALGORITHM WE CAN USE TO MAKE RATINGS THROUGH FEATURE EXTRACTION**

## Feature Extractor

In [1]:
arquivos = [os.path.join(directory, f) for f in sorted(os.listdir(directory))]  

In [1]:
export = 'mouth,pants,shoes,shirt,shorts,sneakers,class\n'  

In [1]:
export

In [1]:
show_images = False
characteristics = []

In [1]:
100 * 200

In [1]:
(2000 / 20000) * 100

In [1]:
for path_image in files:
  
  try:
    original_image = cv2.imread(path_image)
    (H, W) = original_image.shape[:2]
  except:
    continue

  altered_image = original_image.copy()
  image_features = []
  image_name = os.path.basename(os.path.normpath(path_image))
  mouth = pants = shoes = 0
  shirt = shorts = sneakers = 0

  if image_name.startswith('b'): 
    classe = 0
  else:
    classe = 1

  for height in range(0, H):
    for width in range(0, W):
      
      blue = altered_image.item(height, width, 0)
      green = altered_image.item(height, width, 1)
      red = altered_image.item(height, width, 2)

      if (blue >= 95 and blue <= 140 and green >= 160 and green <= 185 and red >= 175 and red <= 205):
        altered_image[height, width] = [0, 255, 255]
        mouth += 1


      if (blue >= 150 and blue <= 180 and green >= 98 and green <= 120 and red >= 0 and red <= 90):
        altered_image[height, width] = [0, 255, 255]
        pants += 1


      if height > (H / 2):
        if (blue >= 25 and blue <= 45 and green >= 25 and green <= 45 and red >= 25 and red <= 45):
          altered_image[height, width] = [0, 255, 255]
          shoes += 1


      if (blue >= 11 and blue <= 50 and green >= 85 and green <= 105 and red >= 240 and red <= 255):
        altered_image[height, width] = [0, 255, 128]
        shirt += 1


      if (blue >= 125 and blue <= 170 and green >= 0 and green <= 12 and red >= 0 and red <= 20):
        altered_image[height, width] = [0, 255, 128]
        shorts += 1


      if height > (H / 2):
        if (blue >= 125 and blue <= 170 and green >= 0 and green <= 12 and red >= 0 and red <= 20):
          altered_image[height, width] = [0, 255, 128]
          sneakers += 1

  boca = round((mouth / (H * W)) * 100, 9)
  calca = round((pants / (H * W)) * 100, 9)
  sapato = round((shoes / (H * W)) * 100, 9)
  camisa = round((shirt / (H * W)) * 100, 9)
  calcao = round((shorts / (H * W)) * 100, 9)
  tenis = round((sneakers / (H * W)) * 100, 9)

  image_features.append(mouth)
  image_features.append(pants)
  image_features.append(shoes)
  image_features.append(shirt)
  image_features.append(shorts)
  image_features.append(sneakers)
  image_features.append(classe)

  characteristics.append(image_features)

    
  f = (",".join([str(item) for item in image_features]))
  export += f + '\n'

  if show_images == True:
    altered_image = cv2.cvtColor(altered_image, cv2.COLOR_BGR2RGB)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    figura, im = plt.subplots(1, 2)
    im[0].imshow(original_image)
    im[0].axis('off')
    im[1].imshow(altered_image)
    im[1].axis('off')
    plt.show()


  #cv2_imshow(original_image)
  #print(H, W)
  #print(image_name)

In [1]:
export

## Training and testing bases

In [1]:
with open('features.csv', 'w') as file:
  for linha in export:
    file.write(linha)
file.closed

In [1]:
dataset = pd.read_csv('features.csv', error_bad_lines = False)

In [1]:
X = dataset.iloc[:, 0:6].values   
X

In [1]:
y = dataset.iloc[:, 6].values  
y

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [1]:
X_train.shape, y_train.shape

In [1]:
X_test.shape, y_test.shape

## Construction and training of the neural network

In [1]:
(6 + 2) / 2

In [1]:
network = tf.keras.Sequential()
network.add(tf.keras.layers.Dense(input_shape = (6,), units = 4, activation='relu'))
network.add(tf.keras.layers.Dense(units=4, activation='relu'))
network.add(tf.keras.layers.Dense(units=4, activation='relu'))
network.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

In [1]:
network.summary()

In [1]:
network.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [1]:
historic = network.fit(X_train, y_train, epochs = 80)

## Neural network evaluation

In [1]:
historic.history.keys()

In [1]:
plt.plot(historic.history['loss']);

In [1]:
plt.plot(historic.history['accuracy']);

In [1]:
X_test.shape

In [1]:
forecasts = network(X_test)
forecasts

In [1]:
forecasts = forecasts > 0.5
forecasts

In [1]:
y_test

In [1]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, forecasts)

In [1]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, forecasts)
cm

In [1]:
sns.heatmap(cm, annot=True);

In [1]:
from sklearn.metrics import classification_report
print(classification_report(y_test, forecasts))

## Upload and rate a single image

In [1]:
network.summary()

In [1]:
image_test = X_test[1]
image_test

In [1]:
image_test.shape

In [1]:
image_test = image_test.reshape(1,-1)
image_test.shape

In [1]:
network.predict(image_test)[0][0]

In [1]:
if network.predict(image_test)[0][0] < 0.5:
  print('Bart')
else:
  print('Homer')

**The case study with the extraction of features has a much better result than just with neural networks.**

https://www.kaggle.com/juniorbueno/neural-networks-simpsons-image-classification

# **If you find this notebook useful, support with an upvote** 👍¶